In [1]:
import tensorflow
import numpy
import transformers
import datasets

print(tensorflow.__version__)
print(numpy.__version__)
print(transformers.__version__)
print(datasets.__version__)

2.6.0
1.21.4
4.11.3
1.14.0


## STEP 1. NSMC 데이터 분석 및 Huggingface dataset 구성
### Huggingface dataset에서 불러오기

In [2]:
import datasets
from datasets import load_dataset
from huggingface_hub import HfApi

huggingface_nsmc_dataset = load_dataset("nsmc")
print(huggingface_nsmc_dataset)


Using custom data configuration default
Reusing dataset nsmc (/aiffel/.cache/huggingface/datasets/nsmc/default/1.1.0/bfd4729bf1a67114e5267e6916b9e4807010aeb238e4a3c2b95fbfa3a014b5f3)


  0%|          | 0/2 [00:00<?, ?it/s]

DatasetDict({
    train: Dataset({
        features: ['id', 'document', 'label'],
        num_rows: 150000
    })
    test: Dataset({
        features: ['id', 'document', 'label'],
        num_rows: 50000
    })
})


- 데이터 크기가 너무 큰가해서, train data 사이즈를 줄여보는 과정(->효과 x)

In [3]:
# Reduce the dataset size by selecting a subset
# For example, to select the first 1000 examples
subset_size = 100000
subset_nsmc_dataset = huggingface_nsmc_dataset["train"].select(list(range(subset_size)))

# Now, subset_nsmc_dataset contains only the first 1000 examples from the original dataset
print(len(subset_nsmc_dataset))

100000


In [4]:
subset_nsmc_dataset[:5]

{'id': ['9976970', '3819312', '10265843', '9045019', '6483659'],
 'document': ['아 더빙.. 진짜 짜증나네요 목소리',
  '흠...포스터보고 초딩영화줄....오버연기조차 가볍지 않구나',
  '너무재밓었다그래서보는것을추천한다',
  '교도소 이야기구먼 ..솔직히 재미는 없다..평점 조정',
  '사이몬페그의 익살스런 연기가 돋보였던 영화!스파이더맨에서 늙어보이기만 했던 커스틴 던스트가 너무나도 이뻐보였다'],
 'label': [0, 1, 0, 0, 1]}

In [5]:
# # Split the original dataset into train, validation, and test
# train_dataset = huggingface_nsmc_dataset['train']
# test_dataset = huggingface_nsmc_dataset['test']

# # You can further split the train_dataset if needed
# #from sklearn.model_selection import train_test_split
# # train_dataset, validation_dataset = train_test_split(train_dataset, test_size=0.2, random_state=42)

In [6]:
train = huggingface_nsmc_dataset['train']
cols = train.column_names
cols

['id', 'document', 'label']

In [7]:
for i in range(5):
    for col in cols:
        print(col, ":", train[col][i])
    print('\n')

id : 9976970
document : 아 더빙.. 진짜 짜증나네요 목소리
label : 0


id : 3819312
document : 흠...포스터보고 초딩영화줄....오버연기조차 가볍지 않구나
label : 1


id : 10265843
document : 너무재밓었다그래서보는것을추천한다
label : 0


id : 9045019
document : 교도소 이야기구먼 ..솔직히 재미는 없다..평점 조정
label : 0


id : 6483659
document : 사이몬페그의 익살스런 연기가 돋보였던 영화!스파이더맨에서 늙어보이기만 했던 커스틴 던스트가 너무나도 이뻐보였다
label : 1




## STEP 2. klue/bert-base model 및 tokenizer 불러오기



이번엔 Huggingface에서 Auto Model과 이에 상응하는 tokenizer을 불러오겠습니다.

In [8]:
# Load model directly
from transformers import AutoTokenizer, AutoModelForMaskedLM, AutoModelForSequenceClassification


tokenizer = AutoTokenizer.from_pretrained("klue/bert-base")
model = AutoModelForMaskedLM.from_pretrained("klue/bert-base", num_labels = 2)

Some weights of the model checkpoint at klue/bert-base were not used when initializing BertForMaskedLM: ['cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


## STEP 3. 위에서 불러온 tokenizer으로 데이터셋을 전처리하고, model 학습 진행해 보기


토크나이징은 transform이라는 함수를 만들고 이전에 만들어두었던 Tokenizer를 사용하는데 이때 dataset의 형태를 확인하고 바꿀 대상을 지정해야 합니다.

nsmc의 경우 'document'가 토크나이징할 대상이므로 data[’document’]로 인덱싱해서 지정합니다.

truncation은 특정 문장이 길어 모델을 다루기 힘들어 질 수 있으므로 짧게 자르는 것을 의미합니다.

return_token_type_ids는 문장이 한개이상일 때 나뉘는걸 보여줍니다. (해당 내용은 task에 필요없으므로 제거합니다)



In [9]:
def transform(data, max_length=64):  # Set your desired max_length here
    return tokenizer(
        data['document'],
        truncation=True,
        padding='max_length',
        max_length=max_length,  # Set the desired maximum sequence length
        return_token_type_ids=False,
    )

In [10]:
#hf_dataset = huggingface_nsmc_dataset.map(transform, batched=True)

# train & test split
hf_train_dataset = subset_nsmc_dataset.map(transform,batched=True)
hf_test_dataset = huggingface_nsmc_dataset['test'].map(transform, batched=True)

  0%|          | 0/100 [00:00<?, ?ba/s]

Loading cached processed dataset at /aiffel/.cache/huggingface/datasets/nsmc/default/1.1.0/bfd4729bf1a67114e5267e6916b9e4807010aeb238e4a3c2b95fbfa3a014b5f3/cache-5a8f99fc2ab0b75e.arrow


데이터셋을 한번에 토크나이징할때 자주 사용하는 기법은 map입니다.

map을 사용하게 되면 Data dictionary에 있는 모든 데이터들이 빠르게 적용시킬 수 있습니다.

우리는 map을 사용해 토크나이징을 진행하기 때문에 batch를 적용해야 되므로 batched=True로 주어야 합니다.

## STEP 4. Fine-tuning을 통하여 모델 성능(accuarcy) 향상시키기
### Trainer를 활용한 학습
Trainer를 사용하기 위해서는 TrainingArguments를 통해 학습 관련 설정을 미리 지정해야 합니다.

In [11]:
import os
import numpy as np
from transformers import Trainer, TrainingArguments

output_dir = "/aiffel/aiffel/transformers"

training_arguments = TrainingArguments(
    output_dir,                                         # output이 저장될 경로
    evaluation_strategy="epoch",           #evaluation하는 빈도
    learning_rate = 2e-5,                         #learning_rate
    per_device_train_batch_size = 512,   # 각 device 당 batch size
    per_device_eval_batch_size = 4,    # evaluation 시에 batch size
    num_train_epochs = 1,                     # train 시킬 총 epochs
    weight_decay = 0.01,                        # weight decay
)

아래에서 생성하게 될 Trainer의 인자로 넘겨주어야 할 것 중에 compute_metrics 메소드가 있습니다.

이것은 task가 classification인지 regression인지에 따라 모델의 출력 형태가 달라지므로 task별로 적합한 출력 형식을 고려해 모델의 성능을 계산하는 방법을 미리 지정해 두는 것입니다.

NSMC 데이터셋은 binary classification에 해당

In [12]:
from datasets import load_metric

metric = load_metric("accuracy")

def compute_metrics(eval_pred):
    predictions,labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    return metric.compute(predictions=predictions, references = labels)

Trainer에 model, arguments, train_dataset, eval_dataset, compute_metrics를 넣고 train을 진행합니다.

In [13]:
#!pip install torch

In [14]:
import gc
gc.collect()

23

In [15]:
trainer = Trainer(
    model=model,           # 학습시킬 model
    args=training_arguments,           # TrainingArguments을 통해 설정한 arguments
    train_dataset=hf_train_dataset,    # training dataset
    #eval_dataset=hf_val_dataset,
    compute_metrics=compute_metrics,
)
trainer.train()
print("슝~")

The following columns in the training set  don't have a corresponding argument in `BertForMaskedLM.forward` and have been ignored: document, id.
***** Running training *****
  Num examples = 100000
  Num Epochs = 1
  Instantaneous batch size per device = 512
  Total train batch size (w. parallel, distributed & accumulation) = 512
  Gradient Accumulation steps = 1
  Total optimization steps = 196


RuntimeError: CUDA out of memory. Tried to allocate 96.00 MiB (GPU 0; 14.76 GiB total capacity; 13.38 GiB already allocated; 3.75 MiB free; 13.50 GiB reserved in total by PyTorch)

In [ ]:
#test 데이터셋으로 평가
trainer.evaluate(hf_test_dataset)

print("완료")

## STEP 5. Bucketing을 적용하여 학습시키고, STEP 4의 결과와의 비교


In [ ]:
from transformers import DataCollatorForLanguageModeling

data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer,
    mlm_probability=0.15,
)

trainer = Trainer(
    model=model,           # 학습시킬 model
    args=training_arguments,           # TrainingArguments을 통해 설정한 arguments
    train_dataset=hf_train_dataset,    # training dataset
    data_collator=data_collator,  # Use the DataCollator
    compute_metrics=compute_metrics,
)


## 회고
- 'RuntimeError: CUDA out of memory' 문제를 해결하기 위해  
: max_length 조절, batch_size 조절, epochs 낮추기, gc.collect() 사용, 데이터 사이즈 줄여보기 등의 여러가지 방법을 시도해보았지만, 결국 훈련을 돌려보지 못했다.
- 그렇지만 오류 덕분에(?) colab과 jupyter notebook을 다 돌려보는 기회가 되었다. 
- 훈련 실행은 실패했지만, huggingface의 모델을 불러와서 task에 적용하는 전체 흐름을 배울 수 있었다.